# Methods of De-treding
This notebook outllines different methods that can be used to de-trend the LongRunMIP and ZECMIP time series.

In [2]:
import nc_time_axis
import warnings

import xarray as xr
import numpy as np
import pandas as pd
from importlib import reload
import string

from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 


import os
import sys

from numpy.typing import ArrayLike
from typing import Optional, Callable, List, Tuple, Dict

warnings.filterwarnings('ignore')

import constants
from constants import PlotConfig
sys.path.append(constants.MODULE_DIR)
import sn_plotting
import xarray_class_accessors as xca
import utils
import stats
import xarray_extender as xce
import signal_to_noise as sn
import open_ds
from classes import ExperimentTypes, LocationsLatLon
import plotting_functions
logger = utils.get_notebook_logger()

In [3]:
notebook_number='05'

In [4]:
import dask.distributed as dd
import tempfile

tempdir = tempfile.TemporaryDirectory("dask-worker-space")
dd.Client(local_directory=tempdir.name, memory_limit='16gb')

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 59.60 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44563,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 59.60 GiB
Comm: tcp://127.0.0.1:33811,Total threads: 4
Dashboard: http://127.0.0.1:44185/status,Memory: 14.90 GiB
Nanny: tcp://127.0.0.1:40807,


# Loading Data

In [5]:
experiment_params = constants.EXPERIMENTS_TO_RUN[0]

In [6]:
control_abrupt_da = open_ds.open_experiment_files(experiment_params, ExperimentTypes.CONTROL)
abrupt4x_da = open_ds.open_experiment_files(experiment_params, ExperimentTypes.ABRUPT4X)



- Opening control
- ['ccsm3' 'cesm104' 'cnrmcm61' 'hadcm3l' 'ipslcm5a' 'mpiesm11' 'mpiesm12']=
- /g/data/w40/ab2313/PhD/longrunmip/tas/regrid_retimestamped=
- Opening files in /g/data/w40/ab2313/PhD/longrunmip/tas/regrid_retimestamped
- --- Running function 'read_longrunmip_netcdf'
- Opening files /g/data/w40/ab2313/PhD/longrunmip/tas/regrid_retimestamped/tas_mon_CCSM3_control_1530_g025.nc
- --- Running function 'read_longrunmip_netcdf'
- Opening files /g/data/w40/ab2313/PhD/longrunmip/tas/regrid_retimestamped/tas_mon_CESM104_control_1000_g025.nc
- --- Running function 'read_longrunmip_netcdf'
- Opening files /g/data/w40/ab2313/PhD/longrunmip/tas/regrid_retimestamped/tas_mon_CNRMCM61_control_2000_g025.nc
- --- Running function 'read_longrunmip_netcdf'
- Opening files /g/data/w40/ab2313/PhD/longrunmip/tas/regrid_retimestamped/tas_mon_HadCM3L_control_1000_g025.nc
- --- Running function 'read_longrunmip_netcdf'
- Opening files /g/data/w40/ab2313/PhD/longrunmip/tas/regrid_retimestamped/t

In [7]:
abrupt4x_da_smean, control_abrupt_da_smean = sn.calculate_global_value(
    abrupt4x_da, control_abrupt_da, experiment_params["variable"])

(None, None)


# Local Points

## SN

In [8]:
def doulbe_trend(da, slice1, slice2, **kwargs):

    trend1_da = stats.trend_fit(da.isel(time=slice(*slice1)),**kwargs)
    trend2_da = stats.trend_fit(da.isel(time=slice(*slice2)),**kwargs)

    return trend2_da.combine_first(trend1_da)

In [9]:
def signal_to_noise_and_save(save_name, *args, **kwargs):
    print(f'Running {save_name}', end='')
    sn_da = sn.signal_to_noise(*args, **kwargs)
    sn_da.to_netcdf(os.path.join(TEST_SAVE_DIR, save_name))
    print(': COMPLETED')

In [10]:
window = constants.WINDOWS_OF_INTEREST[0]

In [11]:
TEST_SAVE_DIR = os.path.join(constants.LONGRUNMIP_DIR, experiment_params['variable'], 'method_test')

In [12]:
for window in constants.WINDOWS_OF_INTEREST[1:]:
    for order in [3,4,5,6]:
        print(f'{window=}, {order=}')
        
        double_trend_da_local = doulbe_trend(abrupt4x_da, (0,100), (100, None),
                                             method='polynomial', order=order)
        detrended_da_local = abrupt4x_da - double_trend_da_local
        
        signal_to_noise_and_save(
            f'abrupt4x_polynomial_static_order_{order}_window_{window}.nc',
            window=window, da=abrupt4x_da,
            da_for_noise=detrended_da_local, rolling_noise=False, time_slice=(0,100))
        
        da_control_no_roll = signal_to_noise_and_save(
            f'control_polynomial_static_order_{order}_window_{window}.nc',
            window=window, da=control_abrupt_da,
            detrend=False, rolling_noise=False)
        
        signal_to_noise_and_save(
              f'abrupt4x_polynomial_rolling_order_{order}_window_{window}.nc', 
            window=window, da=abrupt4x_da,
            da_for_noise=detrended_da_local, rolling_noise=True)
        
        signal_to_noise_and_save(
            f'control_rolling_static_order_{order}_window_{window}.nc', 
            window=window, da=control_abrupt_da,
            detrend=False, rolling_noise=True)

window=80, order=3
Running abrupt4x_polynomial_static_order_3_window_80.nc80, 

MemoryError: Unable to allocate 43.3 GiB for an array with shape (7, 1000, 72, 144, 80) and data type float64

## SN and Bounds

In [13]:
sn_local_multi_static_ds = sn.multiwindow_signal_to_nosie_and_bounds(
    abrupt4x_da.chunk(constants.LONGRUNMIP_CHUNKS),
    control_abrupt_da.chunk(constants.LONGRUNMIP_CHUNKS),
    da_for_noise = detrended_da_local.chunk(constants.LONGRUNMIP_CHUNKS),
    logginglevel='INFO', parallel=False, rolling_noise=False, time_slice=(0, 100),
    windows=constants.WINDOWS_OF_INTEREST[:0])

- sn_kwargs={'windows': [], 'start_window': 21, 'end_window': None, 'step_window': None, 'rolling_noise': False, 'parallel': False, 'time_slice': (0, 100), 'logginglevel': 'INFO'}

Experiment
--------
- windows=[21]
- -- allocate_data_for_noise_calculation
- Dataset for nosie provided
- slicing time with integers (0, 100)
21, - Calculting the rolling signal
- rolling_noise=False
 - Finished

Control
------

- windows=[21]
- -- allocate_data_for_noise_calculation
- Dataset for nosie provided
- slicing time with integers (0, 100)
21, - Calculting the rolling signal
- rolling_noise=False
Calculating bounds
- Calculating Upper and lower control bounds
- Map blocks used
final merge


In [24]:
window = 21

In [27]:
sn_local_multi_static_ds = sn_local_multi_static_ds.compute()

2023-03-06 16:10:35,085 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 9.44 GiB -- Worker memory limit: 14.90 GiB
Exception ignored in: <bound method GCDiagnosis._gc_callback of <distributed.utils_perf.GCDiagnosis object at 0x7fd11b393520>>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/utils_perf.py", line 179, in _gc_callback
    def _gc_callback(self, phase, info):
KeyboardInterrupt: 
Exception ignored in: <bound method GCDiagnosis._gc_callback of <distributed.utils_perf.GCDiagnosis object at 0x7fd11b393520>>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/utils_perf.

- Task exception was never retrieved
future: <Task finished name='Task-1895486' coro=<Client._gather.<locals>.wait() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py:2119> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py", line 2128, in wait
    raise AllExit()
distributed.client.AllExit
- Task exception was never retrieved
future: <Task finished name='Task-1895487' coro=<Client._gather.<locals>.wait() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py:2119> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py", line 2128, in wait
    raise AllExit()
distributed.client.AllExit
- Task exception was never retri

- Task exception was never retrieved
future: <Task finished name='Task-1895503' coro=<Client._gather.<locals>.wait() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py:2119> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py", line 2128, in wait
    raise AllExit()
distributed.client.AllExit
- Task exception was never retrieved
future: <Task finished name='Task-1895504' coro=<Client._gather.<locals>.wait() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py:2119> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py", line 2128, in wait
    raise AllExit()
distributed.client.AllExit
- Task exception was never retri

Exception ignored in: <bound method BaseEventLoop.call_exception_handler of <_UnixSelectorEventLoop running=True closed=False debug=False>>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/asyncio/base_events.py", line 1779, in call_exception_handler
    self.default_exception_handler(context)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/asyncio/base_events.py", line 1753, in default_exception_handler
    logger.error('\n'.join(log_lines), exc_info=exc_info)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/logging/__init__.py", line 1475, in error
    self._log(ERROR, msg, args, **kwargs)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/logging/__init__.py", line 1589, in _log
    self.handle(record)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/logging/__init__.py", line 1599, in handle
    self.callH

- Task exception was never retrieved
future: <Task finished name='Task-1897805' coro=<Client._gather.<locals>.wait() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py:2119> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py", line 2128, in wait
    raise AllExit()
distributed.client.AllExit
- Task exception was never retrieved
future: <Task finished name='Task-1897806' coro=<Client._gather.<locals>.wait() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py:2119> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py", line 2128, in wait
    raise AllExit()
distributed.client.AllExit
- Task exception was never retri

- Task exception was never retrieved
future: <Task finished name='Task-1897824' coro=<Client._gather.<locals>.wait() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py:2119> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py", line 2128, in wait
    raise AllExit()
distributed.client.AllExit
- Task exception was never retrieved
future: <Task finished name='Task-1897825' coro=<Client._gather.<locals>.wait() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py:2119> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/distributed/client.py", line 2128, in wait
    raise AllExit()
distributed.client.AllExit
- Task exception was never retri


KeyboardInterrupt



In [25]:
sn_local_multi_static_ds.to_netcdf(os.path.join(TEST_SAVE_DIR, f'static_window_{window}_double_polynomial.nc'))

2023-03-06 15:28:37,151 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 9.57 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 15:29:51,192 - distributed.nanny - WARNING - Restarting worker
2023-03-06 15:30:07,908 - distributed.nanny - WARNING - Restarting worker
2023-03-06 15:30:09,390 - distributed.nanny - WARNING - Worker process still alive after 3.19999740600586 seconds, killing
2023-03-06 15:30:12,956 - distributed.nanny - WARNING - Restarting worker
2023-03-06 15:30:26,128 - distributed.nanny - WARNING - Worker process still alive after 3.1999980163574224 seconds, killing
2023-03-06 15:45:36,218 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see h

2023-03-06 15:48:54,885 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 10.44 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 15:48:54,973 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 10.44 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 15:48:55,059 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

2023-03-06 15:48:57,308 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 10.45 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 15:48:57,371 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 10.45 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 15:48:57,477 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

2023-03-06 15:48:59,808 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 10.45 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 15:48:59,997 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 10.45 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 15:49:00,080 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

2023-03-06 15:49:02,369 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 10.46 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 15:49:02,467 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 10.46 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 15:49:02,576 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

...

2023-03-06 16:00:38,387 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 12.62 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 16:00:38,458 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 12.62 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 16:00:38,562 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

2023-03-06 16:00:40,884 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 12.62 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 16:00:40,971 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 12.62 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 16:00:41,093 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

2023-03-06 16:00:43,762 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 12.63 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 16:00:44,006 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 12.63 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 16:00:44,062 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

2023-03-06 16:00:46,272 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 11.47 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 16:00:46,377 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 11.47 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 16:00:46,461 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

2023-03-06 16:00:48,780 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 12.80 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 16:00:48,881 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 12.86 GiB -- Worker memory limit: 14.90 GiB
2023-03-06 16:00:48,961 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 